In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the words
words = open('/content/drive/MyDrive/GitHub/makemore_names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:

  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [7]:
C = torch.randn((27, 2))

In [8]:
C[5]

tensor([-0.4690,  0.7029])

In [9]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.4690,  0.7029])

In [10]:
C[X].shape

torch.Size([32, 3, 2])

In [11]:
X[13, 2]

tensor(1)

In [12]:
C[X][13, 2]

tensor([ 2.1039, -2.6619])

In [13]:
C[1]

tensor([ 2.1039, -2.6619])

In [14]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [15]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [16]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [17]:
h

tensor([[-0.7590, -0.9278,  0.3724,  ...,  0.9652, -0.9707, -0.7981],
        [-0.7660, -0.9442,  0.6103,  ...,  0.8738, -0.9034, -0.6670],
        [ 0.2857, -0.9524,  0.2254,  ...,  0.8293, -0.9367, -0.6175],
        ...,
        [ 0.9854, -0.9800,  0.6362,  ...,  0.9989, -0.9993, -0.8264],
        [ 0.5887, -0.9369,  0.9409,  ...,  0.9942, -0.9943, -0.1990],
        [ 1.0000, -0.9979,  0.3407,  ..., -0.5528, -0.9937, -0.7673]])

In [18]:
h.shape

torch.Size([32, 100])

In [19]:
(emb.view(-1, 6) @ W1).shape

torch.Size([32, 100])

In [20]:
b1.shape

torch.Size([100])

In [21]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [22]:
logits = h @ W2 + b2

In [23]:
logits.shape

torch.Size([32, 27])

In [25]:
counts = logits.exp()

In [26]:
prob = counts / counts.sum(1, keepdims=True)

In [27]:
prob.shape

torch.Size([32, 27])

In [30]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(12.6512)